In [1]:
import numpy as np
import sympy as sp

In [3]:
from spe_decompose import decompose_spe

Lamd = 0.4
Gams = 1
Omgs = 1
Zeta = 1

temp = 0.5 # temprature
beta = 1 / temp
npsd = 1 # pade

#w_sp, eta_sp, gamma_sp, beta_sp = sp.symbols(r"\omega, \eta, \gamma, \beta", real=True)
w_sp, lamd_sp, gams_sp, zeta_sp, omgs_sp, beta_sp = sp.symbols(
r"\omega , \lambda, \gamma, \zeta, \Omega_{s}, \beta", real=True)

#phixx_sp = 2 * eta_sp * gamma_sp / (gamma_sp - sp.I * w_sp)
phiyy_sp = 2 * omgs_sp * omgs_sp * lamd_sp / (
omgs_sp * omgs_sp - w_sp * w_sp - zeta_sp * sp.I * w_sp)

spe_vib_sp = phiyy_sp

#sp_para_dict = {eta_sp: eta, gamma_sp: gam}
sp_para_dict = {lamd_sp: Lamd, gams_sp: Gams, omgs_sp: Omgs, zeta_sp: Zeta}

condition_dict = {}
para_dict = {'beta': beta}
etal, etar, etaa, expn = decompose_spe(spe_vib_sp, w_sp, sp_para_dict, para_dict,
                                       condition_dict, npsd)
print(etal)
print(etar)
#print(etaa)
print(expn)

[ 0.49733015+8.19310341e-02j  0.03544993-8.19310341e-02j
 -0.03214094+2.84284380e-18j]
[ 0.03544993+8.19310341e-02j  0.49733015-8.19310341e-02j
 -0.03214094-2.84284380e-18j]
[0.5       +0.8660254j 0.5       -0.8660254j 3.87298335+0.j       ]


In [4]:
#set up spin-boson model
rho_dimension = 1 
dissipaton_modes = npsd + 2
dissipaton_cutoff = 4 # 3 excited states

rho_qubits = rho_dimension * 2 # two times of density matrix in vectorized form
dissipaton_qubits = int(np.ceil(np.log2(dissipaton_cutoff))) * dissipaton_modes # 向上取整
total_qubits = rho_qubits + dissipaton_qubits + 1

H = {'X':1, 'Z':1}
Q = {'Z':1}

alpha = expn.real
omega = expn.imag # γ=α+iΩ
zeta = np.sqrt((etal + etar) / 2)
xi = (etal - etar) / (2j * zeta)
print(alpha)
print(omega)
print(zeta)
print(xi)

[0.5        0.5        3.87298335]
[ 0.8660254 -0.8660254  0.       ]
[0.52206073+0.07846887j 0.52206073-0.07846887j 0.        +0.17927895j]
[-0.06502081-4.32589515e-01j -0.06502081+4.32589515e-01j
 -0.        -1.58570979e-17j]


In [5]:
#evolution parameters
epsilon = 0.05
tau = 0.01

In [6]:
from evolution import Evolution

DQME = Evolution(rho_qubits=rho_qubits, dissipaton_qubits=dissipaton_qubits,
                 modes=dissipaton_modes, cut_off=dissipaton_cutoff,
                 H=H, Q=Q, alpha=alpha, omega=omega, zeta=zeta, xi=xi,
                 epsilon=epsilon, tau=tau)
DQME.construct_UA_circuit()
DQME.construct_US_circuit()
DQME.construct_USdagger_circuit()
display(DQME.UA_circuit.decompose().draw())
display(DQME.US_circuit.decompose().draw())
display(DQME.USdagger_circuit.decompose().draw())

/Users/lix/Nutstore Files/LX2/DQME+QC/codes/DQME_BO/evolution.py:180: ComplexWarning: Casting complex values to real discards the imaginary part
  hermitian_decompsition[label] = float(decomp)


░                    ░           »
q0_0: ─────────────────────────────────────░────────────────────░───────────»
      ┌───┐┌───────────┐┌───┐┌───────────┐ ░                    ░           »
q0_1: ┤ H ├┤ Rz(-0.02) ├┤ H ├┤ Rz(-0.02) ├─░────────────────────░───────────»
      ├───┤└┬──────────┤├───┤└┬──────────┤ ░                    ░           »
q0_2: ┤ H ├─┤ Rz(0.02) ├┤ H ├─┤ Rz(0.02) ├─░────────────────────░────────■──»
      └───┘ └──────────┘└───┘ └──────────┘ ░ ┌───────────────┐  ░ ┌───┐  │  »
q0_3: ─────────────────────────────────────░─┤ Rz(-0.017321) ├──░─┤ H ├──┼──»
                                           ░ ├───────────────┴┐ ░ ├───┤┌─┴─┐»
q0_4: ─────────────────────────────────────░─┤ Rz(-0.0086603) ├─░─┤ H ├┤ X ├»
                                           ░ └┬──────────────┬┘ ░ ├───┤└───┘»
q0_5: ─────────────────────────────────────░──┤ Rz(0.017321) ├──░─┤ H ├─────»
                                           ░ ┌┴──────────────┤  ░ ├───┤     »
q0_6: ─────────────────────────────────────░─┤ Rz(0.0086603) ├──░─┤ H ├─────»
                                           ░ └───────────────┘  ░ └───┘     »
q0_7: ─────────────────────────────────────░────────────────────░───────────»
                                           ░                    ░           »
q0_8: ─────────────────────────────────────░────────────────────░───────────»
                                           ░                    ░           »
«                                                                              »
«q0_0: ────────────────────────────────────────────────────────────────────────»
«                                                                              »
«q0_1: ────────────────────────────────────────────────────────────────────────»
«                                                                              »
«q0_2: ──────────────────■───────────────────────────■──────────────────────■──»
«                        │                           │                      │  »
«q0_3: ──────────────────┼───────────────────────────┼──────────────────────┼──»
«      ┌──────────────┐┌─┴─┐┌───┐┌────────────────┐┌─┴─┐┌────────────────┐┌─┴─┐»
«q0_4: ┤ Rz(0.020172) ├┤ X ├┤ H ├┤ U(π/2,π/2,π/2) ├┤ X ├┤ Rz(-0.0008882) ├┤ X ├»
«      └──────────────┘└───┘└───┘└────────────────┘└───┘└────────────────┘└───┘»
«q0_5: ────────────────────────────────────────────────────────────────────────»
«                                                                              »
«q0_6: ────────────────────────────────────────────────────────────────────────»
«                                                                              »
«q0_7: ────────────────────────────────────────────────────────────────────────»
«                                                                              »
«q0_8: ────────────────────────────────────────────────────────────────────────»
«                                                                              »
«                                                                              »
«q0_0: ────────────────────────────────────────────────────────────────────────»
«                                                                              »
«q0_1: ────────────────────────────────────────────────────────────────────────»
«                                                                              »
«q0_2: ────────■──────────────────────────────────────────■────────────────────»
«            ┌─┴─┐                                      ┌─┴─┐      ┌───┐       »
«q0_3: ──────┤ X ├──────────────■────────────────────■──┤ X ├──────┤ H ├───────»
«      ┌─────┴───┴──────┐┌───┐┌─┴─┐┌──────────────┐┌─┴─┐├───┤┌─────┴───┴──────┐»
«q0_4: ┤ U(π/2,π/2,π/2) ├┤ H ├┤ X ├┤ Rz(0.010442) ├┤ X ├┤ H ├┤ U(π/2,π/2,π/2) ├»
«      └────────────────┘└───┘└───┘└──────────────┘└───┘└───┘└────────────────┘»
«q0_5: ────────────────────────────────────────────────────────────────────────»
«                                                 

┌───────────┐┌───────────────┐┌─────────────┐               »
q50_0: ┤ U3(π,0,π) ├┤ U(0,0,1.5208) ├┤ P(0.000375) ├───────■───────»
       └───────────┘└───────────────┘└─────────────┘       │       »
q50_1: ────────────────────────────────────────────────────┼───────»
                                                           │       »
q50_2: ────────────────────────────────────────────────────┼───────»
                                                           │       »
q50_3: ────────────────────────────────────────────────────┼───────»
                                                    ┌──────┴──────┐»
q50_4: ─────────────────────────────────────────────┤ Rz(0.00025) ├»
                                                    └─────────────┘»
q50_5: ────────────────────────────────────────────────────────────»
                                                                   »
q50_6: ────────────────────────────────────────────────────────────»
                                                                   »
q50_7: ────────────────────────────────────────────────────────────»
                                                                   »
q50_8: ────────────────────────────────────────────────────────────»
                                                                   »
«                     ┌─────────────┐                             »
«q50_0: ──────■───────┤ P(0.000375) ├───────■─────────────■───────»
«             │       └─────────────┘       │             │       »
«q50_1: ──────┼─────────────────────────────┼─────────────┼───────»
«             │                             │             │       »
«q50_2: ──────┼─────────────────────────────┼─────────────┼───────»
«       ┌─────┴──────┐                      │             │       »
«q50_3: ┤ Rz(0.0005) ├──────────────────────┼─────────────┼───────»
«       └────────────┘                      │             │       »
«q50_4: ────────────────────────────────────┼─────────────┼───────»
«                                           │       ┌─────┴──────┐»
«q50_5: ────────────────────────────────────┼───────┤ Rz(0.0005) ├»
«                                    ┌──────┴──────┐└────────────┘»
«q50_6: ─────────────────────────────┤ Rz(0.00025) ├──────────────»
«                                    └─────────────┘              »
«q50_7: ──────────────────────────────────────────────────────────»
«                                                                 »
«q50_8: ──────────────────────────────────────────────────────────»
«                                                                 »
«       ┌──────────────┐                                  ░           »
«q50_0: ┤ P(0.0029047) ├────────■───────────────■─────────░───────────»
«       └──────────────┘        │               │         ░           »
«q50_1: ────────────────────────┼───────────────┼─────────░───────────»
«                               │               │         ░           »
«q50_2: ────────────────────────┼───────────────┼─────────░────────■──»
«                               │               │         ░ ┌───┐  │  »
«q50_3: ────────────────────────┼───────────────┼─────────░─┤ H ├──┼──»
«                               │               │         ░ ├───┤┌─┴─┐»
«q50_4: ────────────────────────┼───────────────┼─────────░─┤ H ├┤ X ├»
«                               │               │         ░ ├───┤└───┘»
«q50_5: ────────────────────────┼───────────────┼─────────░─┤ H ├─────»
«                               │               │         ░ ├───┤     »
«q50_6: ────────────────────────┼───────────────┼─────────░─┤ H ├─────»
«                               │        ┌──────┴───────┐ ░ ├───┤     »
«q50_7: ────────────────────────┼────────┤ Rz(0.003873) ├─░─┤ H ├─────»
«                       ┌───────┴───────┐└──────────────┘ ░ ├───┤     »
«q50_8: ────────────────┤ Rz(0.0019365) ├─────────────────░─┤ H ├─────»
«                       └───────────────┘                 ░ └───┘     »
«                                   

┌────────────────┐┌──────────────┐                               »
q106_0: ┤ U(0,0,-1.5208) ├┤ P(-0.000375) ├───────■───────────────■───────»
        └────────────────┘└──────────────┘       │               │       »
q106_1: ─────────────────────────────────────────┼───────────────┼───────»
                                                 │               │       »
q106_2: ─────────────────────────────────────────┼───────────────┼───────»
                                                 │        ┌──────┴──────┐»
q106_3: ─────────────────────────────────────────┼────────┤ Rz(-0.0005) ├»
                                          ┌──────┴───────┐└─────────────┘»
q106_4: ──────────────────────────────────┤ Rz(-0.00025) ├───────────────»
                                          └──────────────┘               »
q106_5: ─────────────────────────────────────────────────────────────────»
                                                                         »
q106_6: ─────────────────────────────────────────────────────────────────»
                                                                         »
q106_7: ─────────────────────────────────────────────────────────────────»
                                                                         »
q106_8: ─────────────────────────────────────────────────────────────────»
                                                                         »
«        ┌──────────────┐                               ┌───────────────┐»
«q106_0: ┤ P(-0.000375) ├───────■───────────────■───────┤ P(-0.0029047) ├»
«        └──────────────┘       │               │       └───────────────┘»
«q106_1: ───────────────────────┼───────────────┼────────────────────────»
«                               │               │                        »
«q106_2: ───────────────────────┼───────────────┼────────────────────────»
«                               │               │                        »
«q106_3: ───────────────────────┼───────────────┼────────────────────────»
«                               │               │                        »
«q106_4: ───────────────────────┼───────────────┼────────────────────────»
«                               │        ┌──────┴──────┐                 »
«q106_5: ───────────────────────┼────────┤ Rz(-0.0005) ├─────────────────»
«                        ┌──────┴───────┐└─────────────┘                 »
«q106_6: ────────────────┤ Rz(-0.00025) ├────────────────────────────────»
«                        └──────────────┘                                »
«q106_7: ────────────────────────────────────────────────────────────────»
«                                                                        »
«q106_8: ────────────────────────────────────────────────────────────────»
«                                                                        »
«                                            ░                             »
«q106_0: ────────■─────────────────■─────────░───────────────────■─────────»
«                │                 │         ░                   │         »
«q106_1: ────────┼─────────────────┼─────────░───────────────────┼─────────»
«                │                 │         ░                   │         »
«q106_2: ────────┼─────────────────┼─────────░────────■──────────┼─────────»
«                │                 │         ░ ┌───┐  │          │         »
«q106_3: ────────┼─────────────────┼─────────░─┤ H ├──┼──────────┼─────────»
«                │                 │         ░ ├───┤┌─┴─┐┌───────┴────────┐»
«q106_4: ────────┼─────────────────┼─────────░─┤ H ├┤ X ├┤ Rz(-6.278e-05) ├»
«                │                 │         ░ ├───┤└───┘└────────────────┘»
«q106_5: ────────┼─────────────────┼─────────░─┤ H ├───────────────────────»
«                │                 │         ░ ├───┤                       »
«q106_6: ────────┼─────────────────┼─────────░─┤ H ├───────────────────────»
«                │         ┌───────┴───────┐ ░ ├───┤                       »
«q1

In [7]:
print(DQME.UA_circuit.decompose().depth())
print(DQME.US_circuit.decompose().depth())
print(DQME.USdagger_circuit.decompose().depth())

145
238
236


In [8]:
from qiskit.quantum_info import Statevector

initialstate = Statevector.from_label('0'*total_qubits)

In [9]:
steps = 1000
result = DQME.dynamics_run(initial=initialstate, steps=steps)

100%|██████████| 1000/1000 [1:40:06<00:00,  6.01s/it]   


In [10]:
from data import rdo_normal, rho_tilde

rdo_normal_list = []
for i in range(steps+1):
    rdo_normal_list.append(rdo_normal(result[i], total_qubits, rho_qubits))

rdo_normal_dict = {}
for i in range(steps+1):
    label = str(i)       
    rdo_normal_dict[label] = rdo_normal_list[i]
np.savez('rdo_normalized.npz', **rdo_normal_dict)

In [11]:
import matplotlib.pyplot as plt

population = np.zeros(shape=(steps+1), dtype=float)
population_imag = np.zeros(shape=(steps+1), dtype=float)
for i in range(steps+1):
    population[i] = (rdo_normal_list[i][0]-rdo_normal_list[i][3]).real
    population_imag[i] = (rdo_normal_list[i][0]-rdo_normal_list[i][3]).imag
    
plt.plot(np.arange(steps+1) * tau, population, label='population_real', color='blue') # plot of population
plt.plot(np.arange(steps+1) * tau, population_imag, label='population_imag', color='red')
plt.title("Population", fontsize=30, loc='center', color='purple')
plt.legend(loc='best')
plt.xlabel("Time"), plt.ylabel("Population")
plt.savefig('Population.png')
plt.close()